ResNet with Attention

In [3]:
#load keras model
import keras
from keras.models import load_model
import numpy as np
#load model
model = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\ResNet_SE.keras')


In [4]:
import pandas as pd
test_sick = pd.read_pickle(r'C:\Users\Utente\Desktop\Chiara\SUBSET\df_malati_mean_noFMP.pkl')
test_healthy = pd.read_pickle(r'C:\Users\Utente\Desktop\Chiara\SUBSET\df_test_mean.pkl')
train = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_train_mean.pkl")

In [5]:
test_healthy['signals'] = test_healthy['signals'].apply(lambda x: x[0] if isinstance(x, list) and isinstance(x[0], np.ndarray) else x)
test_sick['signals'] = test_sick['signals'].apply(lambda x: x[0] if isinstance(x, list) and isinstance(x[0], np.ndarray) else x)
train['signals'] = train['signals'].apply(lambda x: x[0] if isinstance(x, list) and isinstance(x[0], np.ndarray) else x)

In [6]:
def find_n_windows(l_signal, w_window=3600, Maxoverlap=0.5):
    n_windows = np.ceil(l_signal / w_window)
    if n_windows ==1:
        overlap = 0
    else:
        overlap = np.ceil(((n_windows * w_window) - l_signal) / (n_windows -1))
    if overlap * (n_windows - 1) > (w_window * Maxoverlap):
        n_windows = n_windows -1
        overlap = 0
    return n_windows, overlap


import numpy as np

def WindowSignals(X, y, Maxoverlap=0.5):
    """
    Divide i segnali di X in finestre e assegna a ciascuna finestra l'età gestazionale `y`.
    
    Args:
        X (ndarray): Array di segnali, ciascuno rappresentato come array numpy (dtype=object).
        y (ndarray): Array di etichette (età gestazionale) corrispondenti ai segnali in X (dtype=object).
        Maxoverlap (float): Percentuale massima di sovrapposizione tra le finestre (default 0.5).

    Returns:
        X_post_win (ndarray): Array di finestre dei segnali.
        y_post_win (ndarray): Array di etichette per ciascuna finestra.
    """
    if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
        X = X.to_numpy()  # Conversione a numpy array se X è un DataFrame o Series
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y = y.to_numpy()  # Conversione a numpy array se y è un DataFrame o Series
    

    X_post_win = []
    y_post_win = []

    for i in range(len(X)):  # Itera sugli array in X (ora sono ndarray di tipo object)
        signal = X[i]
        label = y[i]  # L'etichetta associata al segnale

        l_signal = len(signal)  # Lunghezza del segnale
        w_window = 3600  # Lunghezza della finestra

        # Calcola il numero di finestre e la sovrapposizione
        if Maxoverlap > 0:
            n_windows, overlap = find_n_windows(l_signal, w_window, Maxoverlap)
        else:
            n_windows = np.floor(l_signal / w_window)
            overlap = 0

        # Genera le finestre per il segnale corrente
        for n in range(int(n_windows)):
            start_index = int(n * (w_window - overlap))
            end_index = int(start_index + w_window)
            if end_index > l_signal:  # Verifica che la finestra non esca dai limiti
                break
            window = signal[start_index:end_index]
            X_post_win.append(window)
            y_post_win.append(label)  # Aggiungi l'etichetta corrispondente

    return np.array(X_post_win, dtype=object), np.array(y_post_win, dtype=object)


In [7]:
X_windows_train, y_windows_train= WindowSignals(train['signals'], train['y'])
X_windows_test, y_windows_test = WindowSignals(test_healthy['signals'], test_healthy['y'])
X_windows_test_sick, y_windows_test_sick = WindowSignals(test_sick['signals'], test_sick['y'])

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(y_windows_train.reshape(-1, 1))
y_windows_test_scaled = scaler.transform(y_windows_test.reshape(-1, 1))
y_windows_test_sick_scaled = scaler.transform(y_windows_test_sick.reshape(-1, 1))

In [9]:
y_windows_test = y_windows_test_scaled
y_windows_test_sick = y_windows_test_sick_scaled

In [10]:
print(X_windows_test.dtype)
print(X_windows_test_sick.dtype)


object
object


In [11]:
import tensorflow as tf

X_windows_test = tf.convert_to_tensor(X_windows_test, dtype=tf.float32)
X_windows_test_sick = tf.convert_to_tensor(X_windows_test_sick, dtype=tf.float32)
X_windows_test = tf.expand_dims(X_windows_test, axis=-1)
X_windows_test_sick = tf.expand_dims(X_windows_test_sick, axis=-1)

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr, linregress
import numpy as np

def compute_metrics(y_true, y_pred):
    y_true_denorm = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
    y_pred_denorm = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae_denorm = mean_absolute_error(y_true_denorm, y_pred_denorm)
    mse_denorm = mean_squared_error(y_true_denorm, y_pred_denorm)
    mae_denorm_div7 = mae_denorm / 7
    mse_denorm_div7 = mse_denorm / 7
    r2 = r2_score(y_true, y_pred)
    pearson_corr, p_value = pearsonr(y_true, y_pred)
    spearman_corr, _ = spearmanr(y_true, y_pred)
    slope, intercept, r_value, p_linreg, std_err = linregress(y_true, y_pred)
    
    return {
        'MAE_norm': mae,
        'MSE_norm': mse,
        'MAE_denorm': mae_denorm,
        'MSE_denorm': mse_denorm,
        'MAE_denorm_div7': mae_denorm_div7,
        'MSE_denorm_div7': mse_denorm_div7,
        'R2': r2,
        'Pearson': pearson_corr,
        'Spearman': spearman_corr,
        'P_value': p_value,
        'Slope': slope
    }


In [13]:
def bootstrap_metrics(y_true, y_pred, n_bootstrap=1000, random_state=None):
    np.random.seed(random_state)
    n = len(y_true)
    metrics_list = []

    for _ in range(n_bootstrap):
        idx = np.random.choice(range(n), size=n, replace=True)
        metrics = compute_metrics(y_true[idx], y_pred[idx])
        metrics_list.append(metrics)
    
    return metrics_list


In [14]:
def compute_confidence_intervals(metrics_list, alpha=0.05):
    keys = metrics_list[0].keys()
    ci_dict = {}
    for key in keys:
        values = [m[key] for m in metrics_list]
        lower = np.percentile(values, 100 * alpha / 2)
        upper = np.percentile(values, 100 * (1 - alpha / 2))
        mean = np.mean(values)
        ci_dict[key] = (mean, lower, upper)
    
    return ci_dict


In [15]:
y_true_healthy = y_windows_test.flatten()
y_pred_healthy = model.predict(X_windows_test).flatten()
metrics_boot_healthy = bootstrap_metrics(y_true_healthy, y_pred_healthy)
ci_healthy = compute_confidence_intervals(metrics_boot_healthy)

y_true_sick = y_windows_test_sick.flatten()
y_pred_sick = model.predict(X_windows_test_sick).flatten()
metrics_boot_sick = bootstrap_metrics(y_true_sick, y_pred_sick)
ci_sick = compute_confidence_intervals(metrics_boot_sick)


120/120 [==============================] - 6s 49ms/step


In [16]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")


Healthy subjects:
MAE_norm: 0.258 (IC 95%: 0.252 - 0.265)
MSE_norm: 0.106 (IC 95%: 0.102 - 0.112)
MAE_denorm: 8.141 (IC 95%: 7.933 - 8.352)
MSE_denorm: 105.588 (IC 95%: 100.834 - 110.734)
MAE_denorm_div7: 1.163 (IC 95%: 1.133 - 1.193)
MSE_denorm_div7: 15.084 (IC 95%: 14.405 - 15.819)
R2: 0.258 (IC 95%: 0.234 - 0.283)
Pearson: 0.508 (IC 95%: 0.485 - 0.534)
Spearman: 0.500 (IC 95%: 0.474 - 0.525)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.253 (IC 95%: 0.239 - 0.268)

Sick subjects:
MAE_norm: 0.506 (IC 95%: 0.491 - 0.519)
MSE_norm: 0.453 (IC 95%: 0.430 - 0.477)
MAE_denorm: 15.928 (IC 95%: 15.475 - 16.364)
MSE_denorm: 449.620 (IC 95%: 426.606 - 473.373)
MAE_denorm_div7: 2.275 (IC 95%: 2.211 - 2.338)
MSE_denorm_div7: 64.231 (IC 95%: 60.944 - 67.625)
R2: 0.211 (IC 95%: 0.191 - 0.234)
Pearson: 0.653 (IC 95%: 0.635 - 0.670)
Spearman: 0.661 (IC 95%: 0.641 - 0.679)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.212 (IC 95%: 0.204 - 0.221)


In [17]:
lower = np.percentile(y_true_healthy, 5)
upper = np.percentile(y_true_healthy, 95)
print(lower)
print(upper)

mask = (y_true_healthy >= lower) & (y_true_healthy <= upper)
#print(len(y_true), len(y_true_sick), len(y_pred), len(y_pred_sick))
mask_sick = (y_true_sick >= lower) & (y_true_sick <= upper)

filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred = np.array(y_pred_healthy)[mask]
filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick = np.array(y_pred_sick)[mask_sick]

-0.7142857142857135
0.5873015873015888


In [18]:
metrics_boot_healthy_filtered = bootstrap_metrics(filtered_y_true, filtered_y_pred)
ci_healthy_filtered = compute_confidence_intervals(metrics_boot_healthy_filtered)
metrics_boot_sick_filtered = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick)
ci_sick_filtered = compute_confidence_intervals(metrics_boot_sick_filtered)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.227 (IC 95%: 0.220 - 0.233)
MSE_norm: 0.080 (IC 95%: 0.076 - 0.084)
MAE_denorm: 7.136 (IC 95%: 6.943 - 7.324)
MSE_denorm: 79.125 (IC 95%: 75.278 - 82.883)
MAE_denorm_div7: 1.019 (IC 95%: 0.992 - 1.046)
MSE_denorm_div7: 11.304 (IC 95%: 10.754 - 11.840)
R2: 0.189 (IC 95%: 0.157 - 0.220)
Pearson: 0.453 (IC 95%: 0.425 - 0.480)
Spearman: 0.449 (IC 95%: 0.420 - 0.477)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.261 (IC 95%: 0.242 - 0.280)

Sick subjects (filtered):
MAE_norm: 0.256 (IC 95%: 0.249 - 0.263)
MSE_norm: 0.100 (IC 95%: 0.095 - 0.105)
MAE_denorm: 8.067 (IC 95%: 7.845 - 8.293)
MSE_denorm: 99.032 (IC 95%: 93.979 - 103.839)
MAE_denorm_div7: 1.152 (IC 95%: 1.121 - 1.185)
MSE_denorm_div7: 14.147 (IC 95%: 13.426 - 14.834)
R2: 0.224 (IC 95%: 0.191 - 0.258)
Pearson: 0.481 (IC 95%: 0.452 - 0.512)
Spearman: 0.484 (IC 95%: 0.453 - 0.516)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.273 (IC 95%: 0.253 - 0.293)


In [19]:
df_healthy = pd.DataFrame(ci_healthy, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy)
df_sick = pd.DataFrame(ci_sick, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick)
df_healthy_filtered = pd.DataFrame(ci_healthy_filtered, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered)
df_sick_filtered = pd.DataFrame(ci_sick_filtered, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered)

                         Media        IC 2.5%       IC 97.5%
MAE_norm          2.584381e-01   2.518415e-01   2.651383e-01
MSE_norm          1.064123e-01   1.016217e-01   1.115991e-01
MAE_denorm        8.140801e+00   7.933008e+00   8.351857e+00
MSE_denorm        1.055876e+02   1.008341e+02   1.107342e+02
MAE_denorm_div7   1.162972e+00   1.133287e+00   1.193122e+00
MSE_denorm_div7   1.508395e+01   1.440487e+01   1.581918e+01
R2                2.575347e-01   2.339057e-01   2.832260e-01
Pearson           5.080827e-01   4.845255e-01   5.337867e-01
Spearman          4.995963e-01   4.737058e-01   5.247735e-01
P_value          5.059736e-194  3.865079e-257  1.399544e-205
Slope             2.534068e-01   2.387653e-01   2.682488e-01
                      Media     IC 2.5%    IC 97.5%
MAE_norm           0.505656    0.491269    0.519489
MSE_norm           0.453132    0.429938    0.477070
MAE_denorm        15.928176   15.474967   16.363890
MSE_denorm       449.620119  426.606175  473.373034
MAE_deno

Inception Net

In [21]:
model_inc = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\Inception_ResNet_v1.keras')

In [22]:
#y_true_healthy = y_windows_test.flatten()
y_pred_healthy_inc = model_inc.predict(X_windows_test).flatten()
metrics_boot_healthy_inc = bootstrap_metrics(y_true_healthy, y_pred_healthy_inc)
ci_healthy_inc = compute_confidence_intervals(metrics_boot_healthy_inc)

#y_true_sick = y_windows_test_sick.flatten()
y_pred_sick_inc = model_inc.predict(X_windows_test_sick).flatten()
metrics_boot_sick_inc = bootstrap_metrics(y_true_sick, y_pred_sick_inc)
ci_sick_inc = compute_confidence_intervals(metrics_boot_sick_inc)

120/120 [==============================] - 9s 79ms/step


In [23]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")


Healthy subjects:
MAE_norm: 0.301 (IC 95%: 0.293 - 0.308)
MSE_norm: 0.144 (IC 95%: 0.137 - 0.150)
MAE_denorm: 9.481 (IC 95%: 9.238 - 9.717)
MSE_denorm: 142.656 (IC 95%: 136.401 - 148.733)
MAE_denorm_div7: 1.354 (IC 95%: 1.320 - 1.388)
MSE_denorm_div7: 20.379 (IC 95%: 19.486 - 21.248)
R2: -0.004 (IC 95%: -0.010 - -0.001)
Pearson: 0.023 (IC 95%: -0.009 - 0.056)
Spearman: 0.026 (IC 95%: -0.007 - 0.060)
P_value: 0.275 (IC 95%: 0.001 - 0.917)
Slope: 0.000 (IC 95%: -0.000 - 0.001)

Sick subjects:
MAE_norm: 0.625 (IC 95%: 0.607 - 0.641)
MSE_norm: 0.670 (IC 95%: 0.637 - 0.701)
MAE_denorm: 19.685 (IC 95%: 19.124 - 20.198)
MSE_denorm: 664.400 (IC 95%: 632.540 - 695.167)
MAE_denorm_div7: 2.812 (IC 95%: 2.732 - 2.885)
MSE_denorm_div7: 94.914 (IC 95%: 90.363 - 99.310)
R2: -0.166 (IC 95%: -0.190 - -0.143)
Pearson: 0.109 (IC 95%: 0.075 - 0.140)
Spearman: 0.076 (IC 95%: 0.044 - 0.107)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.001 (IC 95%: 0.000 - 0.001)


In [24]:
#filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred_inc = np.array(y_pred_healthy_inc)[mask]
#filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick_inc = np.array(y_pred_sick_inc)[mask_sick]

In [25]:
metrics_boot_healthy_filtered_inc = bootstrap_metrics(filtered_y_true, filtered_y_pred_inc)
ci_healthy_filtered_inc = compute_confidence_intervals(metrics_boot_healthy_filtered_inc)
metrics_boot_sick_filtered_inc = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick_inc)
ci_sick_filtered_inc = compute_confidence_intervals(metrics_boot_sick_filtered_inc)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered_inc.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.256 (IC 95%: 0.249 - 0.262)
MSE_norm: 0.099 (IC 95%: 0.094 - 0.103)
MAE_denorm: 8.049 (IC 95%: 7.846 - 8.253)
MSE_denorm: 97.986 (IC 95%: 93.633 - 102.503)
MAE_denorm_div7: 1.150 (IC 95%: 1.121 - 1.179)
MSE_denorm_div7: 13.998 (IC 95%: 13.376 - 14.643)
R2: -0.004 (IC 95%: -0.009 - -0.000)
Pearson: 0.050 (IC 95%: 0.012 - 0.083)
Spearman: 0.049 (IC 95%: 0.011 - 0.083)
P_value: 0.051 (IC 95%: 0.000 - 0.490)
Slope: 0.001 (IC 95%: 0.000 - 0.001)

Sick subjects (filtered):
MAE_norm: 0.302 (IC 95%: 0.294 - 0.310)
MSE_norm: 0.128 (IC 95%: 0.123 - 0.134)
MAE_denorm: 9.515 (IC 95%: 9.270 - 9.763)
MSE_denorm: 127.460 (IC 95%: 122.159 - 132.792)
MAE_denorm_div7: 1.359 (IC 95%: 1.324 - 1.395)
MSE_denorm_div7: 18.209 (IC 95%: 17.451 - 18.970)
R2: 0.000 (IC 95%: -0.002 - 0.002)
Pearson: 0.053 (IC 95%: 0.013 - 0.094)
Spearman: 0.050 (IC 95%: 0.008 - 0.090)
P_value: 0.062 (IC 95%: 0.000 - 0.531)
Slope: 0.001 (IC 95%: 0.000 - 0.001)


In [26]:
df_healthy_inc = pd.DataFrame(ci_healthy_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_inc)
df_sick_inc = pd.DataFrame(ci_sick_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_inc)
df_healthy_filtered_inc = pd.DataFrame(ci_healthy_filtered_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered_inc)
df_sick_filtered_inc = pd.DataFrame(ci_sick_filtered_inc, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered_inc)

                      Media     IC 2.5%    IC 97.5%
MAE_norm           0.300980    0.293286    0.308474
MSE_norm           0.143770    0.137467    0.149894
MAE_denorm         9.480881    9.238498    9.716934
MSE_denorm       142.655830  136.401449  148.732583
MAE_denorm_div7    1.354412    1.319785    1.388133
MSE_denorm_div7   20.379404   19.485921   21.247512
R2                -0.004413   -0.009666   -0.000686
Pearson            0.022932   -0.009438    0.055613
Spearman           0.026096   -0.006623    0.059689
P_value            0.274821    0.000995    0.916649
Slope              0.000256   -0.000107    0.000621
                      Media       IC 2.5%    IC 97.5%
MAE_norm           0.624932  6.071184e-01    0.641198
MSE_norm           0.669589  6.374806e-01    0.700596
MAE_denorm        19.685342  1.912423e+01   20.197735
MSE_denorm       664.400159  6.325401e+02  695.166809
MAE_denorm_div7    2.812192  2.732033e+00    2.885391
MSE_denorm_div7   94.914308  9.036287e+01   99.30954

ResNet with activity state

In [28]:
model_qa = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\model_with_QA_mean.keras')

In [29]:
train_q_a = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\DockerImage\softmax_predictions_train_non_norm.pkl")
test_q_a = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\DockerImage\softmax_predictions_test_non_norm.pkl")
test_sick_q_a = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\DockerImage\softmax_predictions_test_sick_non_norm.pkl")

In [30]:
train = pd.concat([train, train_q_a], axis=1)
test = pd.concat([test_healthy, test_q_a], axis=1)
test_sick = pd.concat([test_sick, test_sick_q_a], axis=1)

In [31]:
X_windows_train, y_windows_train= WindowSignals(train['signals'], train['y'])
X_windows_test, y_windows_test = WindowSignals(test['signals'], test['y'])
X_windows_test_sick, y_windows_test_sick = WindowSignals(test_sick['signals'], test_sick['y'])

X_windows_train_qa, y_windows_train= WindowSignals(train['softmax_prediction_list'], train['y'])
X_windows_test_qa, y_windows_test_qa = WindowSignals(test['softmax_prediction_list'], test['y'])
X_windows_test_sick_qa, y_windows_test_sick_qa = WindowSignals(test_sick['softmax_prediction_list'], test_sick['y'])

In [32]:
y_windows_test_scaled = scaler.transform(y_windows_test.reshape(-1, 1))
y_windows_test_sick_scaled = scaler.transform(y_windows_test_sick.reshape(-1, 1))
y_windows_test = y_windows_test_scaled
y_windows_test_sick = y_windows_test_sick_scaled

In [33]:
X_test = np.stack([X_windows_test, X_windows_test_qa], axis=-1)  # (batch_size, timesteps, 2)
X_test_sick = np.stack([X_windows_test_sick, X_windows_test_sick_qa], axis=-1)  # (batch_size, timesteps, 2)

In [34]:
X_windows_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
X_windows_test_sick = tf.convert_to_tensor(X_test_sick, dtype=tf.float32)

In [35]:
y_windows_test = tf.convert_to_tensor(y_windows_test, dtype=tf.float32)
y_windows_test_sick = tf.convert_to_tensor(y_windows_test_sick, dtype=tf.float32)

In [36]:
#y_true_healthy = y_windows_test.flatten()
y_pred_healthy_qa = model_qa.predict(X_windows_test).flatten()
metrics_boot_healthy_qa = bootstrap_metrics(y_true_healthy, y_pred_healthy_qa)
ci_healthy_qa = compute_confidence_intervals(metrics_boot_healthy_qa)

#y_true_sick = y_windows_test_sick.flatten()
y_pred_sick_qa = model_qa.predict(X_windows_test_sick).flatten()
metrics_boot_sick_qa = bootstrap_metrics(y_true_sick, y_pred_sick_qa)
ci_sick_qa = compute_confidence_intervals(metrics_boot_sick_qa)

120/120 [==============================] - 5s 45ms/step


In [37]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")


Healthy subjects:
MAE_norm: 0.257 (IC 95%: 0.250 - 0.264)
MSE_norm: 0.105 (IC 95%: 0.100 - 0.110)
MAE_denorm: 8.095 (IC 95%: 7.882 - 8.313)
MSE_denorm: 104.636 (IC 95%: 99.674 - 109.605)
MAE_denorm_div7: 1.156 (IC 95%: 1.126 - 1.188)
MSE_denorm_div7: 14.948 (IC 95%: 14.239 - 15.658)
R2: 0.264 (IC 95%: 0.242 - 0.286)
Pearson: 0.520 (IC 95%: 0.496 - 0.545)
Spearman: 0.517 (IC 95%: 0.494 - 0.541)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.229 (IC 95%: 0.216 - 0.242)

Sick subjects:
MAE_norm: 0.517 (IC 95%: 0.502 - 0.530)
MSE_norm: 0.467 (IC 95%: 0.443 - 0.490)
MAE_denorm: 16.271 (IC 95%: 15.814 - 16.710)
MSE_denorm: 463.082 (IC 95%: 439.626 - 486.128)
MAE_denorm_div7: 2.324 (IC 95%: 2.259 - 2.387)
MSE_denorm_div7: 66.155 (IC 95%: 62.804 - 69.447)
R2: 0.187 (IC 95%: 0.167 - 0.207)
Pearson: 0.663 (IC 95%: 0.645 - 0.681)
Spearman: 0.666 (IC 95%: 0.648 - 0.684)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.186 (IC 95%: 0.179 - 0.193)


In [38]:
#filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred_qa = np.array(y_pred_healthy_qa)[mask]
#filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick_qa = np.array(y_pred_sick_qa)[mask_sick]

In [39]:
metrics_boot_healthy_filtered_qa = bootstrap_metrics(filtered_y_true, filtered_y_pred_qa)
ci_healthy_filtered_qa = compute_confidence_intervals(metrics_boot_healthy_filtered_qa)
metrics_boot_sick_filtered_qa = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick_qa)
ci_sick_filtered_qa = compute_confidence_intervals(metrics_boot_sick_filtered_qa)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered_qa.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.223 (IC 95%: 0.217 - 0.229)
MSE_norm: 0.077 (IC 95%: 0.074 - 0.081)
MAE_denorm: 7.033 (IC 95%: 6.843 - 7.216)
MSE_denorm: 76.542 (IC 95%: 72.958 - 80.443)
MAE_denorm_div7: 1.005 (IC 95%: 0.978 - 1.031)
MSE_denorm_div7: 10.935 (IC 95%: 10.423 - 11.492)
R2: 0.217 (IC 95%: 0.187 - 0.245)
Pearson: 0.468 (IC 95%: 0.438 - 0.496)
Spearman: 0.468 (IC 95%: 0.439 - 0.494)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.237 (IC 95%: 0.220 - 0.254)

Sick subjects (filtered):
MAE_norm: 0.258 (IC 95%: 0.250 - 0.265)
MSE_norm: 0.100 (IC 95%: 0.095 - 0.105)
MAE_denorm: 8.115 (IC 95%: 7.886 - 8.340)
MSE_denorm: 98.949 (IC 95%: 94.074 - 103.911)
MAE_denorm_div7: 1.159 (IC 95%: 1.127 - 1.191)
MSE_denorm_div7: 14.136 (IC 95%: 13.439 - 14.844)
R2: 0.225 (IC 95%: 0.192 - 0.254)
Pearson: 0.475 (IC 95%: 0.442 - 0.506)
Spearman: 0.476 (IC 95%: 0.442 - 0.507)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.231 (IC 95%: 0.214 - 0.248)


In [40]:
df_healthy_qa = pd.DataFrame(ci_healthy_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_qa)
df_sick_qa = pd.DataFrame(ci_sick_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_qa)
df_healthy_filtered_qa = pd.DataFrame(ci_healthy_filtered_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered_qa)
df_sick_filtered_qa = pd.DataFrame(ci_sick_filtered_qa, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered_qa)

                         Media        IC 2.5%       IC 97.5%
MAE_norm          2.569776e-01   2.502251e-01   2.638950e-01
MSE_norm          1.054533e-01   1.004529e-01   1.104611e-01
MAE_denorm        8.094793e+00   7.882092e+00   8.312693e+00
MSE_denorm        1.046360e+02   9.967444e+01   1.096050e+02
MAE_denorm_div7   1.156399e+00   1.126013e+00   1.187528e+00
MSE_denorm_div7   1.494800e+01   1.423921e+01   1.565786e+01
R2                2.636035e-01   2.422369e-01   2.861195e-01
Pearson           5.198943e-01   4.958570e-01   5.446855e-01
Spearman          5.165766e-01   4.935505e-01   5.411034e-01
P_value          7.336343e-206  9.718183e-270  1.052341e-216
Slope             2.286626e-01   2.155601e-01   2.424667e-01
                      Media     IC 2.5%    IC 97.5%
MAE_norm           0.516551    0.502048    0.530483
MSE_norm           0.466699    0.443060    0.489924
MAE_denorm        16.271365   15.814498   16.710201
MSE_denorm       463.082121  439.626433  486.127536
MAE_deno

ResNet with FMP

In [42]:
model_fmp = load_model(r'C:\Users\Utente\Desktop\Chiara\Codici\Cose nuove\Nuovi\Final_mean\Codes\Trained models\model_FMP_mean.keras')

In [43]:
train = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_mean_train_FMP.pkl")
train['FHR'] = train['FHR'].apply(lambda x: x[0] if isinstance(x, list) else x)
train['FMP'] = train['FMP'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [44]:
test = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_mean_test_FMP.pkl")
test['FHR'] = test['FHR'].apply(lambda x: x[0] if isinstance(x, list) else x)
test['FMP'] = test['FMP'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [45]:
test_sick = pd.read_pickle(r"C:\Users\Utente\Desktop\Chiara\SUBSET\df_mean_malati_FMP.pkl")
test_sick['FHR'] = test_sick['FHR'].apply(lambda x: x[0] if isinstance(x, list) else x)
test_sick['FMP'] = test_sick['FMP'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [46]:
X_windows_train_FHR, y_windows_train= WindowSignals(train['FHR'], train['y'])
X_windows_test_FHR, y_windows_test = WindowSignals(test['FHR'], test['y'])
X_windows_test_sick_FHR, y_windows_test_sick = WindowSignals(test_sick['FHR'], test_sick['y'])

In [47]:
X_windows_train_FMP, y_windows_train= WindowSignals(train['FMP'], train['y'])
X_windows_test_FMP, y_windows_test = WindowSignals(test['FMP'], test['y'])
X_windows_test_sick_FMP, y_windows_test_sick = WindowSignals(test_sick['FHR'], test_sick['y'])

In [48]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(y_windows_train.reshape(-1, 1))

y_windows_train_scaled = scaler.transform(y_windows_train.reshape(-1, 1))
y_windows_test_scaled = scaler.transform(y_windows_test.reshape(-1, 1))
y_windows_test_sick_scaled = scaler.transform(y_windows_test_sick.reshape(-1, 1))
y_windows_train = y_windows_train_scaled
y_windows_test = y_windows_test_scaled
y_windows_test_sick = y_windows_test_sick_scaled

In [49]:
X_test = np.stack([X_windows_test_FHR, X_windows_test_FMP], axis=-1)
X_test_sick = np.stack([X_windows_test_sick_FHR, X_windows_test_sick_FMP], axis=-1) 
X_windows_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
X_windows_test_sick = tf.convert_to_tensor(X_test_sick, dtype=tf.float32)

In [50]:
y_windows_test = tf.convert_to_tensor(y_windows_test, dtype=tf.float32)
y_windows_test_sick = tf.convert_to_tensor(y_windows_test_sick, dtype=tf.float32)

In [51]:
#y_true_healthy = y_windows_test.flatten()
y_pred_healthy_fmp = model_fmp.predict(X_windows_test).flatten()
metrics_boot_healthy_fmp = bootstrap_metrics(y_true_healthy, y_pred_healthy_fmp)
ci_healthy_fmp = compute_confidence_intervals(metrics_boot_healthy_fmp)

#y_true_sick = y_windows_test_sick.flatten()
y_pred_sick_fmp = model_fmp.predict(X_windows_test_sick).flatten()
metrics_boot_sick_fmp = bootstrap_metrics(y_true_sick, y_pred_sick_fmp)
ci_sick_fmp = compute_confidence_intervals(metrics_boot_sick_fmp)

120/120 [==============================] - 6s 46ms/step


In [52]:
print("Healthy subjects:")
for metrica, (mean, lower, upper) in ci_healthy_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

print("\nSick subjects:")
for metrica, (mean, lower, upper) in ci_sick_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects:
MAE_norm: 0.255 (IC 95%: 0.248 - 0.262)
MSE_norm: 0.105 (IC 95%: 0.100 - 0.110)
MAE_denorm: 8.035 (IC 95%: 7.817 - 8.246)
MSE_denorm: 104.572 (IC 95%: 99.653 - 109.586)
MAE_denorm_div7: 1.148 (IC 95%: 1.117 - 1.178)
MSE_denorm_div7: 14.939 (IC 95%: 14.236 - 15.655)
R2: 0.264 (IC 95%: 0.243 - 0.285)
Pearson: 0.526 (IC 95%: 0.502 - 0.549)
Spearman: 0.518 (IC 95%: 0.493 - 0.542)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.231 (IC 95%: 0.219 - 0.243)

Sick subjects:
MAE_norm: 0.673 (IC 95%: 0.656 - 0.689)
MSE_norm: 0.736 (IC 95%: 0.698 - 0.780)
MAE_denorm: 21.188 (IC 95%: 20.677 - 21.690)
MSE_denorm: 730.249 (IC 95%: 692.924 - 774.182)
MAE_denorm_div7: 3.027 (IC 95%: 2.954 - 3.099)
MSE_denorm_div7: 104.321 (IC 95%: 98.989 - 110.597)
R2: -0.282 (IC 95%: -0.348 - -0.236)
Pearson: -0.033 (IC 95%: -0.059 - -0.005)
Spearman: 0.028 (IC 95%: -0.005 - 0.057)
P_value: 0.121 (IC 95%: 0.000 - 0.757)
Slope: -0.015 (IC 95%: -0.028 - -0.002)


In [53]:
#filtered_y_true = np.array(y_true_healthy)[mask]
filtered_y_pred_fmp= np.array(y_pred_healthy_fmp)[mask]
#filtered_y_true_sick = np.array(y_true_sick)[mask_sick]
filtered_y_pred_sick_fmp = np.array(y_pred_sick_fmp)[mask_sick]

In [54]:
metrics_boot_healthy_filtered_fmp = bootstrap_metrics(filtered_y_true, filtered_y_pred_fmp)
ci_healthy_filtered_fmp = compute_confidence_intervals(metrics_boot_healthy_filtered_fmp)
metrics_boot_sick_filtered_fmp = bootstrap_metrics(filtered_y_true_sick, filtered_y_pred_sick_fmp)
ci_sick_filtered_fmp = compute_confidence_intervals(metrics_boot_sick_filtered_fmp)
print("Healthy subjects (filtered):")
for metrica, (mean, lower, upper) in ci_healthy_filtered_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")
print("\nSick subjects (filtered):")
for metrica, (mean, lower, upper) in ci_sick_filtered_fmp.items():
    print(f"{metrica}: {mean:.3f} (IC 95%: {lower:.3f} - {upper:.3f})")

Healthy subjects (filtered):
MAE_norm: 0.222 (IC 95%: 0.216 - 0.228)
MSE_norm: 0.078 (IC 95%: 0.074 - 0.081)
MAE_denorm: 6.985 (IC 95%: 6.796 - 7.170)
MSE_denorm: 77.007 (IC 95%: 73.189 - 80.822)
MAE_denorm_div7: 0.998 (IC 95%: 0.971 - 1.024)
MSE_denorm_div7: 11.001 (IC 95%: 10.456 - 11.546)
R2: 0.212 (IC 95%: 0.183 - 0.242)
Pearson: 0.470 (IC 95%: 0.441 - 0.498)
Spearman: 0.468 (IC 95%: 0.439 - 0.496)
P_value: 0.000 (IC 95%: 0.000 - 0.000)
Slope: 0.238 (IC 95%: 0.221 - 0.254)

Sick subjects (filtered):
MAE_norm: 0.399 (IC 95%: 0.387 - 0.412)
MSE_norm: 0.263 (IC 95%: 0.245 - 0.282)
MAE_denorm: 12.570 (IC 95%: 12.177 - 12.978)
MSE_denorm: 260.816 (IC 95%: 242.793 - 280.213)
MAE_denorm_div7: 1.796 (IC 95%: 1.740 - 1.854)
MSE_denorm_div7: 37.259 (IC 95%: 34.685 - 40.030)
R2: -1.047 (IC 95%: -1.202 - -0.896)
Pearson: 0.006 (IC 95%: -0.033 - 0.046)
Spearman: 0.044 (IC 95%: 0.003 - 0.083)
P_value: 0.465 (IC 95%: 0.020 - 0.959)
Slope: 0.006 (IC 95%: -0.031 - 0.045)


In [55]:
df_healthy_fmp = pd.DataFrame(ci_healthy_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_fmp)
df_sick_fmp = pd.DataFrame(ci_sick_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_fmp)
df_healthy_filtered_fmp = pd.DataFrame(ci_healthy_filtered_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_healthy_filtered_fmp)
df_sick_filtered_fmp = pd.DataFrame(ci_sick_filtered_fmp, index=['Media', 'IC 2.5%', 'IC 97.5%']).T
print(df_sick_filtered_fmp)

                         Media        IC 2.5%       IC 97.5%
MAE_norm          2.550658e-01   2.481478e-01   2.617730e-01
MSE_norm          1.053888e-01   1.004316e-01   1.104422e-01
MAE_denorm        8.034573e+00   7.816657e+00   8.245850e+00
MSE_denorm        1.045720e+02   9.965321e+01   1.095862e+02
MAE_denorm_div7   1.147796e+00   1.116665e+00   1.177979e+00
MSE_denorm_div7   1.493886e+01   1.423617e+01   1.565518e+01
R2                2.636915e-01   2.430044e-01   2.847476e-01
Pearson           5.256299e-01   5.021476e-01   5.488820e-01
Spearman          5.177074e-01   4.930397e-01   5.416200e-01
P_value          4.122602e-211  1.018651e-274  4.626781e-223
Slope             2.311272e-01   2.187757e-01   2.431889e-01
                      Media     IC 2.5%    IC 97.5%
MAE_norm           0.672650    0.656419    0.688557
MSE_norm           0.735953    0.698336    0.780229
MAE_denorm        21.188485   20.677204   21.689553
MSE_denorm       730.249168  692.924068  774.181891
MAE_deno